In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

2025-05-13 13:41:18.129302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 13:41:18.587612: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 13:41:18.726682: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 13:41:21.189853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
#loading the model
model = load_model("simple_rnn.h5")
#weights of models
model.get_weights()

2025-05-13 13:41:25.160797: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-05-13 13:41:25.160883: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: archana-Katana-GF76-11UE
2025-05-13 13:41:25.160912: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: archana-Katana-GF76-11UE
2025-05-13 13:41:25.161187: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.144.3
2025-05-13 13:41:25.161250: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.144.3
2025-05-13 13:41:25.161266: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.144.3
2025-05-13 13:41:25.162180: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU inst

[array([[-0.03510298,  0.01683953, -0.07294831, ..., -0.03481156,
         -0.05273194,  0.0528064 ],
        [ 0.00890669, -0.01635015,  0.01378348, ..., -0.04226002,
          0.0318687 , -0.05729134],
        [ 0.03932906, -0.01594858,  0.01159434, ..., -0.06122292,
         -0.01437748,  0.12734683],
        ...,
        [-0.04598498, -0.14627407,  0.00257614, ..., -0.14254774,
         -0.02944726,  0.14543976],
        [ 0.00321687,  0.01198214,  0.06547265, ..., -0.02186427,
         -0.01878053,  0.01342793],
        [-0.02175819, -0.07162098,  0.01573977, ..., -0.08571234,
          0.04732854,  0.02126963]], dtype=float32),
 array([[ 1.53946057e-01, -1.24943040e-01, -2.56501157e-02, ...,
         -5.82456775e-02, -7.40321213e-03,  8.74414071e-02],
        [ 4.30358932e-06,  1.60520151e-02, -6.90768063e-02, ...,
         -3.70165780e-02,  5.82013875e-02,  3.06558404e-02],
        [ 1.55623630e-01,  3.86480428e-02, -3.81772667e-02, ...,
          1.28234088e-01, -3.33981514e-02

In [3]:
max_features = 10000 # Only the top 10,000 words are considered, vocabulary size
max_len = 500 # Maximum review length

# Load word index mapping from IMDB dataset
word_index = imdb.get_word_index()

# Reverse the word index to map integers back to words
reverse_word_index = { value:key for key,value in word_index.items()}

#function to decode the review
def decoded_review(encoded_review):
    return " ".join([reverse_word_index.get(i-3, "?") for i in encoded_review])


#function to preprocess the user input into a padded sequence suitable for model prediction
def preprocessed_text(text, max_features=10000, max_len=500):
    words = text.lower().split()
    
    # Encode each word using the IMDB word index
    encoded_review = [word_index.get(word,2)+3 for word in words] # 2 is for unknown words

    # Pad the sequence to a fixed length
    padded_review = pad_sequences([encoded_review], maxlen=max_len)
    
    # Clip to ensure no index exceeds the embedding input range
    padded_review = padded_review.clip(0, max_features - 1)

    return padded_review

In [4]:
#prediction function
def predict_sentiment(review):
    preprocessed_review = preprocessed_text(review)
    
    prediction = model.predict(preprocessed_review)

    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

    return sentiment, prediction[0][0]

In [5]:
#user input and prediction

example = "the movie was awesom and the acting was great and plot was thrilling even acting was good, connected to the audiance"
sentiment, score = predict_sentiment(example)

print(f"The movie review is: ' {example}', \n The sentiment is: {sentiment} \n prediction score: {score}")

1/1 [==============================] - 1s 722ms/step
The movie review is: ' the movie was awesom and the acting was great and plot was thrilling even acting was good, connected to the audiance', 
 The sentiment is: Positive 
 prediction score: 0.7855156660079956


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Load IMDB dataset for word index and padding
max_features = 10000  # max index value for word embedding
max_len = 500
word_index = imdb.get_word_index()

# Reverse the word index to get word from index
reverse_word_index = {value: key for key, value in word_index.items()}

# Function to decode the review from indices
def decoded_review(encoded_review):
    return " ".join([reverse_word_index.get(i - 3, "?") for i in encoded_review])

# Function to preprocess the user input
def preprocessed_text(text, max_features=10000, max_len=500):
    words = text.lower().split()
    word_index = imdb.get_word_index()  # This should work if you're using the IMDB dataset from Keras
    
    encoded_review = []
    for word in words:
        # Get the word index, but ensure it doesn't exceed max_features-1
        word_idx = word_index.get(word, max_features - 1)  # Map OOV words to max_features-1
        encoded_review.append(word_idx)
    
    # Pad the sequence to make it consistent in length
    padded_review = pad_sequences([encoded_review], maxlen=max_len)
    
    # Ensure all values in padded_review are within the range [0, max_features - 1]
    padded_review = padded_review.clip(0, max_features - 1)
    
    return padded_review

# Example usage for prediction (assuming the model is loaded and compiled)
def predict_sentiment(review, model):
    preprocessed_review = preprocessed_text(review)
    prediction = model.predict(preprocessed_review)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
    return sentiment, prediction[0][0]

# Test
example = "the movie was awesom and the acting was great and plot was thrilling even acting was good, connected to the audiance"
sentiment, score = predict_sentiment(example, model)
print(f"Sentiment: {sentiment}, Score: {score}")

# Decoding a sample encoded review
encoded_example = [word_index.get(word, max_features - 1) for word in example.lower().split()]
decoded = decoded_review(encoded_example)
print("Decoded review:", decoded)


1/1 [==============================] - 0s 109ms/step
Sentiment: Positive, Score: 0.7585527896881104
Decoded review: ? as i edged ? ? life i people ? many i shower no life i edged searching and ? vinod
